# Experiment Setup
This is the setup used for the experiments shown in the project

In [1]:
import numpy as np
import laserhockey.hockey_env as h_env
import gymnasium as gym
from importlib import reload
from TD3_helpers import *
import time
import torch
import DDPG
import TD3
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [2]:
start_config = {
    "name" : "name",
    "agent_type" : "TD3",
    "env_type" : "hockey",
    "test" : False,
    "render" : False,
    "episodes" : 1000,
    "max_steps" : int(1e5),
    "mode" : "normal",
    "eps" : 0.1,
    "discount":0.99,
    "update_target_every":100,
    "update_policy_every":2,
    "hidden_sizes_actor" : [256,256],
    "hidden_sizes_critic" : [256,256],
    "iter_fit" : 1,
    "batch_size" : 256,
    "smoothing_std"  : 0.0001,
    "smoothing_clip" : 0.0002,
    "checkpoint1" : None,
    "checkpoint2" : None,
    "learning_rate_critic": 0.001,
    "learning_rate_actor": 0.001,
    "buffer_size" : int(1e6),
    "theta" : 0.005,
    "prio_replay" : False,
    "exp_phase" : 0,
    "cdq" : True,
    "ou" : False
}
# lr of 0.0001 for both seems to work best for hockey
# lr 0.001 for pendulum
# iter_fit 40 for walker, 20 for rest
# eps 0.1 seems to be best
# discount 1 shows best results (winning later isn't worse than earlier?)
# 20 iterations with policy delay 2 best for hockey
# ou noise better for hockey
###
# N noise
# prio replay
# lr 0.0005 for both

In [7]:
### WIN/LOSS/DRAW EVALUATION OF AGENTS IN HOCKEY - WEAK ###
for agent_type in ["TD3", "DDPG", "DPU", "TPS", "CDQ"]:
    print(agent_type)
    config = start_config.copy()
    config["mode"] = "weak"
    #config["agent_type"] = "TD3"
    config["name"] = "fixedactionscompare"
    config["checkpoint1"] = f'./results/{config["agent_type"]}_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["test"] = True
    init_train(config)

TD3


Testing...:  12%|██████▊                                                 | 122653/1000000 [01:11<08:29, 1722.00steps/s]


Wins: 507
Losses: 269
Draws: 224
W/L: 1.8847583643122676
DDPG


Testing...:  12%|██████▌                                                 | 116720/1000000 [01:04<08:04, 1822.07steps/s]


Wins: 504
Losses: 299
Draws: 197
W/L: 1.68561872909699
DPU


...

Wins: 469
Losses: 296
Draws: 235
W/L: 1.5844594594594594
TPS


Testing...:  12%|██████▋                                                 | 118487/1000000 [00:59<07:24, 1981.29steps/s]


Wins: 494
Losses: 299
Draws: 207
W/L: 1.6521739130434783
CDQ


Testing...:  12%|██████▉                                                 | 124578/1000000 [01:06<07:44, 1884.72steps/s]

Wins: 477
Losses: 294
Draws: 229
W/L: 1.6224489795918366


In [ ]:
### WEAK EXPERIMENT ###
for agent_type in ["TD3", "DDPG", "DPU", "TPS", "CDQ"]:
    print("AGENT", agent_type)
    
    config["discount"] = 0.99
    config["ou"] = False
    config["test"] = False
    config["learning_rate_critic"] = 0.001
    config["learning_rate_actor"] = 0.001
    config["smoothing_std"] = 0.0001
    config["smoothing_clip"] = 0.0002
    config["exp_phase"] = int(2e3)
    config["episodes"] = 1000
    config["max_steps"] = int(2e5)
    config["agent_type"] = agent_type
    config["env_type"] = "hockey"
    config["mode"] = "weak"
    config["prio_replay"] = True
    config["name"] = "hockey_rerun"
    init_train(config)
    config["checkpoint1"] = f'./results/{config["agent_type"]}_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["test"] = True
    init_train(config)

In [ ]:
### PENDULUM EXPERIMENT ###
for agent_type in ["TD3", "DDPG", "DPU", "TPS", "CDQ"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["discount"] = 0.99
    config["episodes"] = 50
    config["max_steps"] = int(1e4)
    config["exp_phase"] = 0
    config["hidden_sizes_critic"] = [32,32]
    config["hidden_sizes_actor"] = [32,32]
    config["learning_rate_critic"] = 0.001
    config["learning_rate_actor"] = 0.001
    config["agent_type"] = agent_type
    config["env_type"] = "pendulum"
    config["name"] = "pendulum_rerun"
    init_train(config)

In [ ]:
### CHEETAH EXPERIMENT ###
for agent_type in ["DDPG", "TD3", "DPU", "TPS", "CDQ"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["discount"] = 0.99
    config["episodes"] = 200
    config["exp_phase"] = 2000
    config["max_steps"] = int(1e5)
    config["learning_rate_critic"] = 0.001
    config["learning_rate_actor"] = 0.001
    config["agent_type"] = agent_type
    config["env_type"] = "cheetah_rerun"
    config["name"] = "agentcompare"
    init_train(config)

##### WALKER EXPERIMENT ###
for agent_type in ["CDQ", "TD3"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["episodes"] = 500
    config["exp_phase"] = 50
    config["max_steps"] = int(2e5)
    config["learning_rate_critic"] = 0.001
    config["learning_rate_actor"] = 0.001
    config["agent_type"] = agent_type
    config["env_type"] = "walker"
    config["name"] = "walker_rerun"
    init_train(config)
    config["checkpoint1"] = f'./results/{config["agent_type"]}_{config["env_type"]}_{config["name"]}_{config["mode"]}_agent.pth'

# AFTER THIS NOT USED IN PROJECT!!!

In [ ]:
### NORMAL EXPERIMENT ###
for agent_type in ["TD3", "DDPG", "DPU", "TPS", "CDQ"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["discount"] = 0.99
    config["learning_rate_critic"] = 0.001
    config["learning_rate_actor"] = 0.001
    config["exp_phase"] = 50
    config["episodes"] = 1000
    config["agent_type"] = agent_type
    config["env_type"] = "hockey"
    config["mode"] = "normal"
    config["name"] = f'agentcomparefinal'
    init_train(config)
    config["checkpoint1"] = f'./results/{config["agent_type"]}_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["test"] = True
    init_train(config)

In [ ]:
### DEFENSE EXPERIMENT ###
for agent_type in ["TD3", "DDPG", "DPU", "TPS", "CDQ"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["episodes"] = 1000
    config["agent_type"] = agent_type
    config["env_type"] = "hockey"
    config["mode"] = "defense"
    config["name"] = "agentcompare_new"
    init_train(config)
    config["checkpoint1"] = f'./results/{config["agent_type"]}_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["test"] = True
    init_train(config)

In [ ]:
### ATTACK EXPERIMENT ###
for agent_type in ["DDPG", "DPU", "TPS", "CDQ", "TD3"]:
    print("AGENT", agent_type)
    config = start_config.copy()
    config["episodes"] = 1000
    config["agent_type"] = agent_type
    config["env_type"] = "hockey"
    config["mode"] = "attack"
    config["name"] = f'env_{config["env_type"]}_agent_{agent_type}'
    init_train(config)
    config["checkpoint1"] = f'./results/{config["agent_type"]}_hockey_{config["name"]}_{config["mode"]}_agent.pth'

In [ ]:
### priority replay !!!NOT USED!!! ###
for pri in [True, False]:
    config = start_config.copy()
    config["episodes"] = 5000
    config["prio_replay"] = pri
    config["name"] = f"prio_{pri}"
    config["mode"] = "normal"
    agent_type="TD3"
    env_type="hockey"
    init_train(agent_type, env_type, config)
    config["checkpoint1"] = f'./results/{agent_type}_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    # test agent
    config["mode"] = "normal"
    config["test"] = True
    #config["episodes"] = 10
    #config["render"] = True
    init_train(agent_type, env_type, config)

In [ ]:
### NOT USED ###
config = start_config.copy()
config["episodes"] = 1000
config["name"] = "20ktest"
for i in range(1):
    if i>0:
        config["mode"] = "selfplay"
    config["checkpoint1"] = f'./results/TD3Agent_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["checkpoint2"] = f'./results/TD3Agent_hockey_{config["name"]}_{config["mode"]}_agent.pth'
    config["name"] = f"selfplay{i}"
    config["mode"] = "selfplay"
    config["test"] = False
    init_train(config)
    config["test"] = True
    config["mode"] = "normal"
    config["checkpoint2"] = None
    init_train(config)